<a href="https://colab.research.google.com/github/AnkitaDasData/AnalyticsHub/blob/main/Projects/LLM_with_Semantic_Search/Rerank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ReRank

## Setup

Load needed API keys and relevant Python libaries.

In [1]:
 !pip install cohere
 !pip install weaviate-client
 !pip install python-dotenv

In [2]:
import os
import json
import cohere
import weaviate
from dotenv import load_dotenv, find_dotenv
#_ = load_dotenv(find_dotenv()) # read local .env file

In [3]:
# Step 3: Connecting
#library of functions that use LLM called via API
# Set your API key here
os.environ['WEAVIATE_API_KEY'] = '76320a90-53d8-42bc-b41d-678647c6672e'
os.environ['COHERE_API_KEY'] = 'pEyGu0nvKEqSeqFYTc6VP0YVlaYcOi7yzZ3e3IMI'
co = cohere.Client(os.environ['COHERE_API_KEY'])

In [4]:
# Step 3: Set up the connection to the Weaviate instance (no API key)
# Set the API key
# Set your API key here
# Initialize client with your API key
client = weaviate.Client(
    url="https://cohere-demo.weaviate.network",
    auth_client_secret=weaviate.AuthApiKey(api_key=os.getenv('WEAVIATE_API_KEY')),
    additional_headers={"X-Cohere-Api-Key": os.getenv('COHERE_API_KEY')}
)

# Example query (replace "YourClassName" and fields with the relevant class and fields)
#result = client.query.get("YourClassName", ["field1", "field2"]).do()

#print(result)

<ipython-input-4-155bff1b5ec2>:5: DeprecationWarning: 
Python client v3 `weaviate.Client(...)` connections and methods are deprecated and will
            be removed by 2024-11-30.

            Upgrade your code to use Python client v4 `weaviate.WeaviateClient` connections and methods.
                - For Python Client v4 usage, see: https://weaviate.io/developers/weaviate/client-libraries/python
                - For code migration, see: https://weaviate.io/developers/weaviate/client-libraries/python/v3_v4_migration

            If you have to use v3 code, install the v3 client and pin the v3 dependency in your requirements file: `weaviate-client>=3.26.7;<4.0.0`
  client = weaviate.Client(


## Dense Retrieval

In [5]:
# Performs a dense retrieval search.
#     Args:
#         query: The search query.
#         properties: A list of properties to retrieve, or a single property name as a string.
#                     If None, all properties will be retrieved.
#         num_results: The maximum number of results to retrieve.
#     Returns:
#         A list of results.

def dense_retrieval(query,
                    #results_lang='en',
                    properties=["title", "url", "text"],
                    num_results=3):

    # Define nearText for dense retrieval
    nearText = {
        "concepts": [query],  # Querying based on concepts (dense retrieval)
        #"certainty": 0.7        # Optional: Only return results with a similarity > 0.7
        "distance": 0.8  # Adjust this value to control the search radius
    }

    # # Filter by language
    # where_filter = {
    #     "path": ["lang"],
    #     "operator": "Equal",
    #     "valueString": results_lang
    # }

    # If properties is a single string, convert it to a list
    if isinstance(properties, str):
        properties = [properties]

    # Execute the query with nearText
    response = (
        client.query
        .get("Articles", properties)
        .with_near_text(nearText)
        #.with_where(where_filter)
        .with_limit(num_results)
        .do()
    )

    # Print the raw response for debugging
    #print("Raw response:", json.dumps(response, indent=4))

    # Extract the results from the response
    result = response['data']['Get']['Articles']
    return result
    #return response.do()


In [6]:
query = "What is the capital of Canada?"

In [7]:
dense_retrieval_results = dense_retrieval(query)

In [8]:
def print_result(results):
    if results:
        for i, result in enumerate(results):
            print(f"Result {i+1}:")
            for key, value in result.items():
                print(f"  {key}: {value}")
            print()  # Empty line between results
    else:
        print("No results found.")

In [9]:
print_result(dense_retrieval_results)

Result 1:
  text: The governor general of the province had designated Kingston as the capital in 1841. However, the major population centres of Toronto and Montreal, as well as the former capital of Lower Canada, Quebec City, all had legislators dissatisfied with Kingston. Anglophone merchants in Quebec were the main group supportive of the Kingston arrangement. In 1842, a vote rejected Kingston as the capital, and study of potential candidates included the then-named Bytown, but that option proved less popular than Toronto or Montreal. In 1843, a report of the Executive Council recommended Montreal as the capital as a more fortifiable location and commercial centre, however, the Governor General refused to execute a move without a parliamentary vote. In 1844, the Queen's acceptance of a parliamentary vote moved the capital to Montreal.
  title: Ottawa
  url: https://en.wikipedia.org/wiki?curid=22219

Result 2:
  text: For brief periods, Toronto was twice the capital of the united Prov

## Improving Keyword Search with ReRank

In [10]:
# Define the keyword_search function
def keyword_search(query,
                   results_lang='en',
                   properties=["title", "url", "text"],
                   num_results=3):
    """
    Searches for articles based on the query and returns results.
    Args:
        query: The search query string.
        results_lang: Language filter for the results.
        properties: List of properties to retrieve for each article.
        num_results: Number of results to return.
    Returns:
        List of search results.
    """
    where_filter = {
        "path": ["lang"],
        "operator": "Equal",
        "valueString": results_lang
    }

    # Serialize and deserialize the filter to avoid threading issues
    where_filter_json = json.dumps(where_filter)
    where_filter = json.loads(where_filter_json)

    # Perform the query (replace `client` with your configured Weaviate client)
    response = (
        client.query.get("Articles", properties)
        .with_bm25(query=query)
        .with_where(where_filter)
        .with_limit(num_results)
        .do()
    )

    # Extract the results
    result = response['data']['Get']['Articles']
    return result

In [11]:
# Function to print results
def print_results(results, reranked=False):
    """
    Prints the details of the results, including relevance scores.
    Args:
        results: List of dictionaries containing the results.
        reranked: Boolean, set to True if printing reranked results.
    """
    header = "Reranked Results" if reranked else "Original Results"
    print(f"\n--- {header} ---\n")
    for i, result in enumerate(results):
        title = result.get('title', 'No Title')
        text = result.get('text', 'No Text')
        # Check if '_additional' exists and is a dictionary before accessing 'distance'
        score = result.get('score', 'No Score')  # First, try getting the regular 'score' field
        #if score == 'No Score' and isinstance(result.get('_additional'), dict):
        #    score = result.get('_additional', {}).get('distance', 'No Score')
        print(f"i:{i}")
        print(f"Title: {title}")
        print(f"Text: {text[:300]}...")  # Truncated for brevity
        print(f"Relevance Score: {score}")
        print("-" * 50)

In [12]:
# Function to prepare results for reranking
def prepare_for_reranking(results):
    """
    Prepare a list of documents from the results for reranking.
    Args:
        results: List of dictionaries containing search results.
    Returns:
        List of dictionaries containing title, text, and scores for reranking.
    """
    documents = [
        {
            "text": result.get("text", "No Text"),
            "title": result.get("title", "No Title"),
            #"score": result.get('score', result.get('_additional', {}).get('distance', 'No Score'))
        }
        for result in results
    ]
    return documents


In [13]:
# Function to rerank results based on the query
def rerank_results(query, documents, top_n=5):
    """
    Reranks the given documents based on the query.
    Args:
        query: The search query string.
        documents: List of document texts to be reranked.
        top_n: Number of top results to return.
    Returns:
        List of reranked results with their scores.
    """
    # Simulate reranking (replace this with actual API/model call)
    reranked = [
        {"title": doc["title"], "text": doc["text"], "score": len(doc["text"])}  # Example: score by text length
        for doc in documents
    ]
    reranked = sorted(reranked, key=lambda x: x['score'], reverse=True)[:top_n]
    return reranked

In [14]:
query_1 = "What is the capital of Canada?"

# Fetch original results (ensure `client` is correctly configured for Weaviate)
results = keyword_search(
    query_1,
    results_lang='en',
    properties=["text", "title", "url", "views", "lang", "_additional {distance}"],
    num_results=10
    )

# Print original results
print_results(results)

# Prepare documents for reranking
documents = prepare_for_reranking(results)

# Print prepared results with relevance scores
print("\n--- Prepared Documents for Reranking ---\n")
for i, doc in enumerate(documents):
    print(f"i:{i}")
    print(f"Title: {doc['title']}")
    print(f"Text: {doc['text'][:300]}...")  # Truncated for brevity
    #print(f"Relevance Score: {doc['score']}")
    print("-" * 50)

# Perform reranking
reranked_results = rerank_results(query_1, documents, top_n=5)

# Print reranked results with relevance scores
print_results(reranked_results, reranked=True)


--- Original Results ---

i:0
Title: Monarchy of Canada
Text: In his 1990 book, "Continental Divide: the Values and Institutions of the United States and Canada," Seymour Martin Lipset argues that the presence of the monarchy in Canada helps distinguish Canadian identity from American identity. Since at least the 1930s, supporters of the Crown have held the op...
Relevance Score: No Score
--------------------------------------------------
i:1
Title: Early modern period
Text: North America outside the zone of Spanish settlement was a contested area in the 17th century. Spain had founded small settlements in Florida and Georgia but nowhere near the size of those in New Spain or the Caribbean islands. France, The Netherlands, and Great Britain held several colonies in Nort...
Relevance Score: No Score
--------------------------------------------------
i:2
Title: Flag of Canada
Text: By the Second World War, the Red Ensign was viewed as Canada's "de facto" national flag. A joint committee

In [15]:
query_2 = "Who is the tallest person in history?"

# Fetch original results (ensure `client` is correctly configured for Weaviate)
results = keyword_search(
    query_2,
    results_lang='en',
    properties=["text", "title", "url", "views", "lang", "_additional {distance}"],
    num_results=10
    )

# Print original results
print_results(results)

# Prepare documents for reranking
documents = prepare_for_reranking(results)

# Print prepared results with relevance scores
print("\n--- Prepared Documents for Reranking ---\n")
for i, doc in enumerate(documents):
    print(f"i:{i}")
    print(f"Title: {doc['title']}")
    print(f"Text: {doc['text'][:300]}...")  # Truncated for brevity
    #print(f"Relevance Score: {doc['score']}")
    print("-" * 50)

# Perform reranking
reranked_results = rerank_results(query_2, documents, top_n=5)

# Print reranked results with relevance scores
print_results(reranked_results, reranked=True)


--- Original Results ---

i:0
Title: Pejorative
Text: When a term begins as pejorative and eventually is adopted in a non-pejorative sense, this is called "melioration" or "amelioration". One example is the shift in meaning of the word "nice" from meaning a person was foolish to meaning that a person is pleasant. When performed deliberately, it is desc...
Relevance Score: No Score
--------------------------------------------------
i:1
Title: Ages of consent in Europe
Text: Prior to the 1922 independence of the Irish Free State, the law in Ireland was that of the United Kingdom of Great Britain and Ireland (see the UK history section). Anal sex was illegal under the Offences against the Person Act 1861, while the Criminal Law Amendment Act 1885 criminalised "Defilement...
Relevance Score: No Score
--------------------------------------------------
i:2
Title: History of Japan
Text: The population of Japan peaked at 128,083,960 in 2008. It had decreased by 2,373,960 by December 2020. In 